# 랜덤 포레스트
- 여러개의 결정 트리 생성하여 그 결과를 결합하여 예측
- Bagging(Bootstrap Aggregating)라는 방식을 사용해 각 트리를 독립적으로 학습시키고, 투표나 평균을 통해 최종 예측을 만듬.
- 분류, 회귀에서 매우 높은 성능을 낸다.
- 과적합을 방지하고 일반화 성능을 향상시키는데 효과적임.

## 부트스트랩 샘플: 중복을 허용한 샘플링 기법
- 확률은 분류문제에서, 여러 결정 트리로 부터 각 클래스에 속할 확률을 추정
- 예측은 회귀문제에서, 각 결정트리가 예측한 값을 평균하여 최종 예측값을 구함.

## 훈련방법
- 분류모델에서는 전체 특성개수의 제곱근만큼 특성선택
- 회귀모델에서는 전체특성 다 사용

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine= pd.read_csv('https://bit.ly/wine_csv_data')

data= wine[['alcohol', 'sugar', 'pH']].to_numpy()
target=wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf= RandomForestClassifier(n_jobs= -1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
                                                       # cross_validate()함수에서 훈련세트의 점수도 함께 반환하도록 설정

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
                                                      # 교차검증 과정에서 모델이 test data set에 대해 평가한 정확도

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_) # 각 feature의 중요도

[0.23167441 0.50039841 0.26792718]


In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
                            #(out of back)은 샘플링과정에서 샘플링되지않은 남은 샘플을 의미
                            #oob점수를 사용하면 교차검증을 대신 할 수 있어 결과적으로 훈련세트에 더 많은 샘플을 사용할 수 있다.
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


#엑스트라 트리
랜덤 포레스트와 유사한 앙상블 기법
분할 방식:
- 각 노드에서 전체 특성 중 일부를 무작위로 선택

랜덤포레스트와 차이점:
- 부트스트랩 사용 x 각 결정 트리는 전체 훈련세트사용
노드 분할할 때, 가장 좋은 분할을 찾는 것이 아닌 무작위 분할
- 무작위 분할로 트리구성이 빠름



In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et= ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그래디언트 부스팅(gradient boosting)
- 여러 개의 약한 예측모델, 주로 결정 트리를 결합하여 강력한 예측모델을 만듬.
- 실무에서 많이 사용
- 모델이 반복적으로 학습하며 잔여오차를 줄여가는 방식으로 작동함
- 초기 모델이 예측한 값에서 발생하는 오차를 보완하기 위해 새로운 모델을 계속 추가하는 방식.
- 새로운 결정트리가 분할할 때 사용하는 방식은 결정트리와 동일함. 다만 손실함수의 기울기를 감소하는 목표로 학습.

## 알고리즘
- 경사 하강법을 사용하여 트리를 연속적으로 앙상블에 추가
- 분류에서는 로지스틱 손실함수 사용, 회귀에서는 평균 제곱 오차 함수를 사용함.
- 결정 트리를 연속해서 추가하며 손실함수가 가장 낮은 곳으로 이동한다. 즉 깊이가 얕은 트리를 사용하면서 손실함수를 최소화하는 방법

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gd= GradientBoostingClassifier(random_state=42)
scores= cross_validate(gd, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gd= GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
                              # 트리갯수         # learning_rate는 통상 0.1부터 시작
scores= cross_validate(gd, train_input, train_target,return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gd.fit(train_input, train_target)
print(gd.feature_importances_)

[0.15887763 0.6799705  0.16115187]


# 히스토그램 기반 그레디언트 부스팅(Histogram-based Gradient Boosting)
- 연속형 특성 값을 구간(bin)으로 나누어 히스토그램을 생성하므로, 계산량이 적어지게 되어 계산 효율성을 높인 방법
- 특히 대용량 데이터에서 학습속도를 크게 향상시키는 특징이 있다.
- 훈련데이터를 각 특성의 값 256개 정수 구간(bin)으로 나누고, 각각의 구간을 학습하는 훈련.(256개 중 1개는 어떤 샘플에 누락 값이 있으면 이를 사용하기위해 남겨둠)
- 노드를 분할할 때, 특성의 범위가 제한되어있다.
- 즉, 각 특성을 미리 설정된 구간 단위로만 분할할 수 있기 때문에 분할 기준이 제한된다는 의미. 최적의 분할을 매우 빠르게 찾을 수 있음.

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb= HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
result= permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923